# speciesgrids demo

This notebook demonstrates querying the speciesgrids data product using geopandas and duckdb. The examples use the `s3://obis-products/speciesgrids/h3_7/` remote datasource as default. To query a local copy instead, use the local file path.

## Regional species list using duckdb

In the example below we use duckdb to query the gridded product with H3 resolution 7 to obtain a regional species list. The [geometry for our region of interest](https://wktmap.com/?b40bc054) is encoded as WKT and used in the duckdb query.

In [2]:
import duckdb
import pyarrow.dataset as ds

wkt = "POLYGON ((2.694397 51.187951, 2.694397 51.271367, 3.013 51.271367, 3.013 51.187951, 2.694397 51.187951))"

con = duckdb.connect()
con.sql("""
    install spatial;
    load spatial;
""")
dataset = ds.dataset("s3://obis-products/speciesgrids/h3_7/", format="parquet")
con.register("dataset", dataset)

df = con.execute(f"""
	select
        species,
        sum(records) as records,
        min(min_year) as min_year,
        max(max_year) as max_year,
        max(source_obis) as source_obis,
        max(source_gbif) as source_gbif
    from dataset
    where st_intersects(st_geomfromwkb(geometry), st_geomfromtext('{wkt}')) 
    group by species
    order by sum(records) desc
""").fetchdf()
df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,species,records,min_year,max_year,source_obis,source_gbif
0,Larus argentatus,1118392.0,1837.0,2024.0,True,True
1,Larus fuscus,84912.0,1964.0,2024.0,True,True
2,Larus canus,11298.0,1953.0,2024.0,True,True
3,Larus ridibundus,9528.0,1911.0,2024.0,True,True
4,Phalacrocorax carbo,8610.0,1910.0,2024.0,True,True
...,...,...,...,...,...,...
1620,Wandonia haliotis,1.0,2015.0,2015.0,False,True
1621,Raja undulata,1.0,2000.0,2000.0,False,True
1622,Akashiwo sanguinea,1.0,2021.0,2021.0,True,False
1623,Pelagostrobilidium paraepacrum,1.0,2014.0,2014.0,False,True


## Species distributions using geopandas

This example uses geopandas to query the gridded product with H3 resolution 7 to obtain species distributions for the genus Abra, and lonboard to visualize the distributions on a map.

In [4]:
import geopandas
import lonboard
import seaborn as sns

filters = [("genus", "==", "Abra")]
gdf = geopandas.read_parquet("s3://obis-products/speciesgrids/h3_7/", filters=filters)[["cell", "records", "geometry", "species"]]

def generate_colors(unique_species):
    palette = sns.color_palette("Paired", len(unique_species))
    rgb_colors = [[int(r*255), int(g*255), int(b*255)] for r, g, b in palette]
    color_map = dict(zip(unique_species, rgb_colors))
    colors = lonboard.colormap.apply_categorical_cmap(gdf["species"], color_map)
    return colors

point_layer = lonboard.ScatterplotLayer.from_geopandas(gdf)
point_layer.get_radius = 10000
point_layer.radius_max_pixels = 2
point_layer.get_fill_color = generate_colors(gdf["species"].unique())
lonboard.Map([point_layer])